# Description of Problem ( the Challenge ) ( from Kaggle )
## The Challenge
* It’s difficult to fathom just how vast and diverse our natural world is.
* There are over 5,000 species of mammals, 10,000 species of birds, 30,000 species of fish – and astonishingly, over 400,000 different types of flowers.
* In this competition, you’re challenged to build a machine learning model that identifies the type of flowers in a dataset of images (for simplicity, we’re sticking to just over 100 types).
## TPUs
* TPUs are powerful hardware accelerators specialized in deep learning tasks. They were developed (and first used) by Google to process large image databases, such as extracting all the text from Street View. This competition is designed for you to give TPUs a try.
* TPU quotas are available on Kaggle at no cost to users.

# Source Notebooks ( thanks!! ) -
* Competition - Learn how to use Tensor Processing Units (TPUs) on Kaggle
 * https://www.kaggle.com/competitions/tpu-getting-started
* Petals to the Metal - Flower Classification on TPU
 * 975.4s - TPU v3-8 Public Score - 0.90724
 * https://www.kaggle.com/code/shivam2111/petals-to-the-metal-flower-classification-on-tpu
* A Beginner's TPU Kernel (Single Model, 0.97) 
 * https://www.kaggle.com/code/chankhavu/a-beginner-s-tpu-kernel-single-model-0-97

# Evaluation Basis
* Submissions are evaluated on **macro F1 score ( higher is better )**.
 * **F1** = 2 * ( precision * recall ) / ( precision + recall ) or Harmonic Mean of Precision and Recall = 1 / ( ( 1/Recall + 1/Precision ) / 2 )
 * **precision** = TruePositive / ( TruePositive + FalsePositive )
 * **recall** = TruePositive / ( TruePositive + FalseNegative )
 * **Cross-Entropy Loss** = 
* 2022.Nov.30 - Leaderboard #1 **0.98509** ( Marek Nurzynski ) #49 **0.09724** ( ~this source notebook )

# Versions ( most recent at top )
* 14 - **0.94241** 2583.4s ( ~40m ) - TPU v3-8
 * 512x512 images, warm-up of model
 * Warm-up 10 EPOCH to val_loss: 0.9334 (on SparseCategoricalCrossentropy) val_sparse_categorical_accuracy: 0.8050
 * Epoch 20 ( at 21m 55s )
   * loss: 0.0096 - sparse_categorical_accuracy: 0.9966 - categorical_accuracy: 0.0210 - f1: 0.0191 - custom_f1: 1.2450
   * val_loss: 0.2230 - val_sparse_categorical_accuracy: 0.9526 - val_categorical_accuracy: 0.0210 - val_f1: 0.0189 - val_custom_f1: 1.4303
   * validation: 0.951
* 5 - first submission **0.92293** ( 45 of 100+ ) 936.3s - TPU v3-8
 * model = DenseNet201 - Trainable params: 18,292,712
 * train = 40 Epochs, early-stopped on epoch 27 val_sparse_categorical_accuracy 0.9313
* 1 - base notebook, ? 0.90724 975.4s ( = 15+m )

# To Do
* more custom TPU structures
 * Detailed guide to custom training with TPUs
   * https://www.kaggle.com/code/yihdarshieh/detailed-guide-to-custom-training-with-tpus/notebook
* Handle over-fitting ( training becomes perfect but validation stays at some level )
 * add dropout
 * augment the source images more
 * train more on the edge conditions
* Color Analysis
 * given picture, determine the N most common (? representative ) colors
   * https://stackoverflow.com/questions/3241929/python-find-dominant-most-common-color-in-an-image
* use psuedo-labeling ( & incremental training ) to resolve the easy ones & focus on the hard ones
 * training should focus efforts on the more-esoteric after easy are done
 * unknowns should be classified in phases from most-known to least
* use RAPIDS SVR ( uses TPU )
* Transform images from RGB to **HSV** or other color space
 * then to color spectrum (histogram) vector ( which are the most common colors ( hues ) )
* Transform images from RGB (+HSV) via 2D FFT & then to 'frequency distribution' vector & ? then to frequency main
 * sound analog - https://towardsdatascience.com/understanding-audio-data-fourier-transform-fft-spectrogram-and-speech-recognition-a4072d228520
 * lower-frequency towards center
 * orientation of frequency = orientation of 2D FFT
   * we want to be agnostic so we can integrate/average over circles
* just use pre-trained model resultant embeddings, don't bother re-training/tuning existing models ( that are large )
* make stand-alone (convolutional) model that doesn't use pre-trained ( ? from dog & cats notebook )
* use curve-fit to predict quality of model ( as function of effort = epochs and/or time )
* visualize layer weights & biases to gain understanding ( ? from dog & cats notebook )
* implement Weights & Balances ( wandb )
* compare performance ( speed ) on CPU vs. GPU vs. TPU
 * and how to optimize use, initial runs of TPU only use fraction
* Data Augmentation via:
 * CutMix - https://arxiv.org/abs/1905.04899
   * mix 2 images via regions and also the labels
 * MixUp - https://arxiv.org/abs/1710.09412
   * mix 2 images via blend and also the labels

In [ ]:
# kaggle API use here?
# https://github.com/Kaggle/kaggle-api
# creating Kaggle Dataset using Kaggle API
#!pip install --upgrade --force-reinstall --no-deps kaggle

# from kaggle.api.kaggle_api_extended import KaggleApi
# api = KaggleApi()
# api.authenticate()
# api.competition_download_file('sentiment-analysis-on-movie-reviews', 'train.tsv.zip', path='./')
# api.competition_download_file('sentiment-analysis-on-movie-reviews', 'test.tsv.zip', path='./')

# !kaggle competitions list -s health  # Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.

# kaggle competitions {list, files, download, submit, submissions, leaderboard}
# kaggle datasets {list, files, download, create, version, init}
# kaggle kernels {list, init, push, pull, output, status}
# kaggle config {view, set, unset}
!kaggle datasets list -m
# upload a dataset using the following API 
#!kaggle datasets create -p /home/jupyter/commonlit/commonlit-v0/


In [ ]:
# Run-Mode - Dev ( highly iterative, just to see if things work ) , Test ( general method but not maximal runtime ), Prod/Run ( prob. not EDA, lots of CPU/Memory )
# Accelerator ( CPU, GPU or TPU )
# 
class CFG:
    USE_TENSORBOARD = False
    wandb = False
    competition = 'Petals to the Metal - Flower Classification on TPU'
    _wandb_kernel = 'nadeau-multi'
    model = "Multi"
    DO_EDA = False
    #FEATURE_SELECT = 'Middle'  # 'min', 'all'
    DO_KERAS = True
    #DO_LogisticRegression = False
    #DO_Catboost = False
    #DO_LightAutoML = False
    
N_THREADS = 4
RANDOM_STATE = 123
    
def class2dict(f):
    return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

# Importing Dependencies

In [ ]:
# good to have lightweight start cell to show that notebook is running
from datetime import datetime as dt
import pytz
tz_NY = pytz.timezone('America/New_York')

import time
time_start = time.time()

print( "Local (NY) Time: ", dt.now( tz_NY ).strftime("%Y-%m-%d %H:%M:%S") )

In [ ]:
import pandas as pd
print( 'pd.__version__ =', pd.__version__ ) # pd.__version__ = 1.3.2
pd.set_option('display.max_rows', 300)  # or 1000
pd.set_option('display.max_columns', 500)  # or 1000
pd.set_option('display.width', 500)  # or 1000
pd.set_option('display.max_colwidth', None )  # or 199

import numpy as np 
import re
import math

import sklearn
print( 'sklearn.__version__ =', sklearn.__version__ )  # v 0.23.2 is back-rev, expect 1.02.+
#from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_fscore_support

import matplotlib.pyplot as plt

import tensorflow as tf
print( "Tensorflow version " + tf.__version__ ) # Tensorflow version 2.4.1
from tensorflow_addons.metrics import F1Score

#import kernel_tensorflow_utils as ktu  # for HardwareInfo, ImageTransform, LRSchedulers # exists as kernel_tensorflow_utils.py in Input/Utility Scripts directory 

In [ ]:
import random 
def set_seed( x: int=RANDOM_STATE, threads: int=N_THREADS ) -> None: 
    """Sets seed for results reproducibility."""
    random.seed(x)
    np.random.seed(x)
#     torch.manual_seed(x)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False
#     torch.set_num_threads(threads)
#     if torch.cuda.is_available(): torch.cuda.manual_seed_all(x)
    tf.random.set_seed(x)

set_seed()

# Monitor via Weights & Balances ( wandb )

In [ ]:
if CFG.wandb :  # CFG.wandb:
    # wandb
    !rm -rf ./wandb/
    !pip install --upgrade -q wandb
    import wandb
    print('wandb.__version__', wandb.__version__)
    from wandb.integration.keras import WandbCallback
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        #api_key = user_secrets.get_secret('wandb_key')
        #wandb.login( key = api_key )
        secret_value_0 = user_secrets.get_secret("wandb_api")
        wandb.login( key = secret_value_0 )
        anonymous = None
        # 1. Start a new wandb_run
        # At the top of your training script, start a new run       
        wandb_run = wandb.init(
            settings=wandb.Settings(start_method="thread"), # For versions prior to 0.13.0 we suggest using
            project='Classification-Flowers',
            entity = 'tnadeau',
            name = 'Classification-Multi',
            notes = 'Notes go here.',
            config = class2dict(CFG),
            group = 'GPU',
            job_type = "train",
            anonymous = anony,
            reinit = True  # to enable multiple 'runs' from a single script
        )
        # Capture a dictionary of hyperparameters with config
        wandb.config = {
                "n_estimators": 500, # default = 100
                "max_depth": 4, # default = 3
                "min_samples_split": 5,  # default = 2
                "learning_rate": 0.01,  # def = 0.01
                "loss": "squared_error", # def = 'squared_error'
            }
        
    except:
        wandb.login( anonymous = 'must' )
        print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your \
               W&B access token. Use the Label name as WANDB. \nGet your W&B access \
               token from here: https://wandb.ai/authorize')

# Setup TPU and Distribution strategy

In [ ]:
!nvidia-smi
# GPU T4 x2 - NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4 - 2 x Tesla T4
# GPU P100 - NVIDIA - Tesla P100-PCIE...

In [ ]:
# Detect TPU, return appropriate distribution strategy

# for GPU something like:
# https://www.kaggle.com/code/landlord/numba-cuda-mandelbrot
# from numba import cuda
# from number import * 
# new_func = cuda.jit( restype =, argtypes = , device = True)(old_func)
# @cuda.jit( argtypes = []) def new_func
# ? cuda.grid(2)
# cuda.to_device( image )


try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print( 'Running on TPU .master() =', tpu.master())  # Running on TPU  grpc://10.0.0.2:8470
    print( 'cluster_spec()', tpu.cluster_spec() )  # ClusterSpec({'worker': ['10.0.0.2:8470']})
    print( '.get_job_name()', tpu.get_job_name())
    print( '.get_master()', tpu.get_master())
    print( '.get_tpu_system_metadata()', tpu.get_tpu_system_metadata())
    print( '.num_accelerators()', tpu.num_accelerators())
    print( '.task_id', tpu.task_id)
    print( '.task_type', tpu.task_type) # worker
except ValueError:
    tpu = None

gpus = tf.config.experimental.list_physical_devices('GPU')
num_gpus = len(gpus)    

if tpu:
    tf.config.experimental_connect_to_cluster( tpu )
    tf.tpu.experimental.initialize_tpu_system( tpu )
    strategy = tf.distribute.experimental.TPUStrategy( tpu )
elif gpus :
    print( 'Running on GPU(s)', num_gpus )
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth( device = gpu, enable = True )
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(num_gpus, "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
    if num_gpus == 0:
        strategy = tf.distribute.OneDeviceStrategy(device='CPU')
        print("Setting strategy to OneDeviceStrategy(device='CPU')")
    elif num_gpus == 1:  # Accelerator = GPU P100
        strategy = tf.distribute.OneDeviceStrategy(device='GPU')
        print("Setting strategy to OneDeviceStrategy(device='GPU')")
    else:
        strategy = tf.distribute.MirroredStrategy()
        print("Setting strategy to MirroredStrategy()")  # seems to run out of memory
else:
    strategy = tf.distribute.get_strategy() 

print( "REPLICAS: ", strategy.num_replicas_in_sync )
# REPLICAS:  8 ? TPU 1VM v3-8
# REPLICAS:  2 GPU T4 x 2
# REPLICAS:  1 CPU, GPU P100 
# if CPU ( not GPU/TPU ) probably do not 'augment' images

# Mixed Precision and/or XLA¶
The following booleans can enable mixed precision and/or XLA on GPU/TPU. By default TPU already uses some mixed precision but we can add more. These allow the GPU/TPU memory to handle larger batch sizes and can speed up the training process. The Nvidia V100 GPU has special Tensor Cores which get utilized when mixed precision is enabled. Unfortunately Kaggle's Nvidia P100 GPU does not have Tensor Cores to receive speed up.

In [ ]:
MIXED_PRECISION = True
XLA_ACCELERATE = True

if MIXED_PRECISION:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

# Run-Time Configuration

In [ ]:
# Helper variables
AUTO = tf.data.experimental.AUTOTUNE  # tf.data.AUTOTUNE
IMAGE_SHAPE = [512,512]   #  [192,192], [512,512] # ?this does 'resize', no instead seems to pick a sub-set of images & work from there
if num_gpus > 1 :
    BATCH_SIZE = 16
else :
    BATCH_SIZE = 16 * strategy.num_replicas_in_sync # for CPU is only 1 but should be 4, for GPU P100 is 1, for TPU is 8

WARMUP_EPOCHS = 20
TRAIN_EPOCHS = 30
RANDOM_SEED = 42

# Data Processing

I am not quite exerienced with the tfrecord and efficient tpu optimization so I spent a huge amout of time on the cloud local error 😅😅😅😅 Then I googled and realised that TPUs cannot access local data they only use cloud verified data and therefore in next cell i am going to get the same data but from google cloud.
**am still learning ig**

In [ ]:
from kaggle_datasets import KaggleDatasets
# google cloud store path
# Data dirs


PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(PATH)
# gs://kds-7187323d9756797ee580f506f69177097d6bce252b2e21ea5ccb7a01

data_dir_by_size = { # available image sizes
    (192, 192): '/tfrecords-jpeg-192x192',
    (224, 224): '/tfrecords-jpeg-224x224',
    (331, 331): '/tfrecords-jpeg-331x331',
    (512, 512): '/tfrecords-jpeg-512x512'
}

GCS_PATH_SELECT = { # available image sizes
    192: PATH + '/tfrecords-jpeg-192x192',
    224: PATH + '/tfrecords-jpeg-224x224',
    331: PATH + '/tfrecords-jpeg-331x331',
    512: PATH + '/tfrecords-jpeg-512x512'
}
subdir = data_dir_by_size[ ( IMAGE_SHAPE[0], IMAGE_SHAPE[1] ) ]

GCS_PATH = GCS_PATH_SELECT[IMAGE_SHAPE[0]]
print(GCS_PATH)
# gs://kds-7187323d9756797ee580f506f69177097d6bce252b2e21ea5ccb7a01
# gs://kds-7187323d9756797ee580f506f69177097d6bce252b2e21ea5ccb7a01/tfrecords-jpeg-512x512

In [ ]:
# tfrec files contain multiple data types at once ( the image, the label, etc. )
# tf.io.gfile.glob(   pattern )
TRAINING_IMAGES = tf.io.gfile.glob( f'{GCS_PATH}/train/*.tfrec' ) # glob() returns a list of files that match the given patterns
VALID_IMAGES = tf.io.gfile.glob( f'{GCS_PATH}/val/*.tfrec' )
TEST_IMAGES = tf.io.gfile.glob( f'{GCS_PATH}/test/*.tfrec' )

# Extending the dataset with additional data
# added dataset from: https://www.kaggle.com/datasets/kirillblinov/tf-flower-photo-tfrec
ext_gcs = None
#ext_gcs = KaggleDatasets().get_gcs_path('tf-flower-photo-tfrec')  # not found  Needs to be in local input directory

if ( ext_gcs ) :
    imagenet_files = tf.io.gfile.glob(ext_gcs + '/imagenet' + subdir + '/*.tfrec')  # (GCS_PATH_EXT + '/imagenet_no_test/tfrecords-jpeg-{size}x{size}/*.tfrec'.format(size=SIZE))
    inaturelist_files = tf.io.gfile.glob(ext_gcs + '/inaturalist' + subdir + '/*.tfrec')
    openimage_files = tf.io.gfile.glob(ext_gcs + '/openimage' + subdir + '/*.tfrec')
    oxford_files = tf.io.gfile.glob(ext_gcs + '/oxford_102' + subdir + '/*.tfrec')
    tensorflow_files = tf.io.gfile.glob(ext_gcs + '/tf_flowers' + subdir + '/*.tfrec')

    TRAINING_IMAGES = TRAINING_IMAGES + imagenet_files + inaturelist_files + openimage_files + oxford_files + tensorflow_files

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen',         'watercress',       'canna lily',            # 80 - 89
           'hippeastrum',      'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 103

print( len( CLASSES ) ) # 104

In [ ]:
def count_files(filenames):
    '''Count number of files in the dataset ( actually file system )'''
    n = [int(re.compile(r"-([0-9]*)\.").search(file).group(1)) for file in filenames]
    return np.sum(n)

In [ ]:
NUM_TRAIN_IMG = count_files( TRAINING_IMAGES )
NUM_VALID_IMG = count_files( VALID_IMAGES )
NUM_TEST_IMG  = count_files( TEST_IMAGES )
print(f'Number of training images : {NUM_TRAIN_IMG} \nNumber of validation images : {NUM_VALID_IMG} \nNumber of unlabeled test images : {NUM_TEST_IMG}')

# Number of training images : 12753 ( = ~ 16 * 798 = 12768 )
# with extended dataset this moves up to 68094 training images
# Number of validation images : 3712 
# Number of unlabeled test images : 7382

# with additional tf-flower-photo-tfrec, counts for train increases
# Number of training images : 68094 # increases > 5-fold, does distribution change?
# Number of validation images : 3712 
# Number of unlabeled test images : 7382

In [ ]:
# do more EDA
# what is histogram of Train & Valid
# are there any errors

# Dataset Helper Functions

In [ ]:
def decode_image( img ):
    '''Load Image From The Dataset ( and convert to TF-ready floating point )'''
    image = tf.io.decode_jpeg( img, channels = 3 )
    image = tf.cast( image, tf.float32 ) / 255.0  # tf.float32 doesn't need to be tf.float32 since original data is only int8
    image = tf.reshape( image, [ *IMAGE_SHAPE, 3 ] )
    return image

def onehot( image, label ):
    return image, tf.one_hot( label, len(CLASSES) )

In [ ]:
# define tfr-record parsing functions
def read_labeled_tfrecord( example ):  # example: A string tensor representing a `tf.train.Example`
    '''Read Labeled tfrecord'''
    labeled_struct = {
        'image': tf.io.FixedLenFeature([],tf.string),
        'class': tf.io.FixedLenFeature([],tf.int64)
    }
    parsed = tf.io.parse_single_example( example, labeled_struct)
    image = decode_image( parsed['image'] )
    label = tf.cast( parsed['class'], tf.int32 )  # tf.int32 - only 104 different values so could be int16
    return image, label

def read_unlabeled_tfrecord( example ):
    '''Read unlabeled tfrecord'''
    unlabeled_struct = {
        'image': tf.io.FixedLenFeature([],tf.string),
        'id': tf.io.FixedLenFeature([],tf.string)
    }
    parsed = tf.io.parse_single_example( example, unlabeled_struct)
    image = decode_image( parsed['image'] )
    idnum = parsed['id']
    return image, idnum

In [ ]:
def load_dataset( filenames, is_labeled = True, inorder = False ):
    '''Load the tfrecord as Dataset.
    is_label(bool) : is the data labeled or unlabeled
    inorder(bool) : Should the data be inorder or loaded as soon as it arrives'''
    options = tf.data.Options()
    if not inorder:
        options.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset( filenames, num_parallel_reads = AUTO )
    dataset = dataset.with_options( options )
    dataset = dataset.map( read_labeled_tfrecord if is_labeled else read_unlabeled_tfrecord )
    return dataset

In [ ]:
def data_aug( image, label ):
    '''Image Augummentation'''
    image = tf.image.random_flip_left_right( image ) 
    return image, label

In [ ]:
# order should possibly be:  .cache(), .shuffle(), .repeat(), .map(), .filter(), .batch(), .prefetch()
# our order is: .map(), .repeat(), .shuffle(), .batch(), [.cache()], .prefetch()
def get_train_dataset( do_aug = True, do_repeat = True, do_shuffle = True, do_onehot = False ):
    '''Load the training dataset'''
    dataset = load_dataset( TRAINING_IMAGES, is_labeled=True)
    dataset = dataset.map( data_aug, num_parallel_calls = AUTO )
    if do_onehot:
        dataset = dataset.map(onehot, num_parallel_calls=AUTO)
    if do_aug: dataset = dataset.map( transform, num_parallel_calls = AUTO )
    if do_repeat: dataset = dataset.repeat()
    if do_shuffle: dataset = dataset.shuffle( 10 * BATCH_SIZE  )  # 49 shuffle(   buffer_size, seed=None, reshuffle_each_iteration=None, name=None )
    dataset = dataset.batch( BATCH_SIZE )  # could instead be num_parallel_calls = tf.data.AUTOTUNE
    # dataset = dataset.cache()
    dataset = dataset.prefetch( buffer_size = AUTO )
    return dataset

def get_train_dataset_preview( ordered = True, do_aug = True, do_repeat = True, do_shuffle = True, do_onehot = False ):
    '''Load the training dataset'''
    dataset = load_dataset( TRAINING_IMAGES, is_labeled=True, inorder = ordered )
    if do_onehot:
        dataset = dataset.map(onehot, num_parallel_calls=AUTO)
    #dataset = dataset.map( data_aug, num_parallel_calls = AUTO )
    #if do_aug: dataset = dataset.map( transform, num_parallel_calls = AUTO )
    #if do_repeat: dataset = dataset.repeat()
    #if do_shuffle: dataset = dataset.shuffle( 10 * BATCH_SIZE  )  # 49 shuffle(   buffer_size, seed=None, reshuffle_each_iteration=None, name=None )
    dataset = dataset.batch( BATCH_SIZE )  # could instead be num_parallel_calls = tf.data.AUTOTUNE
    dataset = dataset.cache()
    dataset = dataset.prefetch( buffer_size = AUTO )
    return dataset

#2022-12-02 14:14:08.461447: W tensorflow/core/kernels/data/cache_dataset_ops.cc:757] The calling iterator did not fully read the dataset being cached.
# In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset will be discarded.
# This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
def get_valid_dataset( ordered = False, do_onehot = False ):
    '''Load the validation dataset'''
    dataset = load_dataset( VALID_IMAGES, is_labeled=True, inorder=ordered)
    if do_onehot:
        dataset = dataset.map(onehot, num_parallel_calls=AUTO)
    dataset = dataset.batch( BATCH_SIZE )
    dataset = dataset.cache() # perhaps this should be in front of .batch()
    dataset = dataset.prefetch( buffer_size = AUTO)
    return dataset

def get_test_dataset( ordered = False ):
    '''Load the test dataset'''
    dataset = load_dataset( TEST_IMAGES, is_labeled=False, inorder=ordered)
    dataset = dataset.batch( BATCH_SIZE)
    # dataset = dataset.cache()
    dataset = dataset.prefetch( buffer_size = AUTO)
    return dataset

In [ ]:
import tensorflow.keras.backend as K
# alternatively use from keras.preprocessing.image import ImageDataGenerator
# tf.keras.preprocessing.image_dataset_from_directory(
# img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#     rotation_range=54
#     , width_shift_range=0.15
#     , height_shift_range=0.15
#     , brightness_range=None
#     , zoom_range=[1.0, 1.25]
#     , fill_mode='constant'
#     , horizontal_flip=True
#     , preprocessing_function=None
# ) 
# image = img_gen.random_transform(image)

#@staticmethod
def get_mat( rotation, shear, height_zoom, width_zoom, height_shift, width_shift ) :
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos( shear )
    s2 = tf.math.sin( shear )
    shear_matrix = tf.reshape( tf.concat( [ one, s2, zero, zero,c2,zero, zero,zero,one],axis = 0 ),[ 3,3 ] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))


def transform( image, label ) :  # why is label passed & returned?
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SHAPE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')

    #shr = 5. * tf.random.normal( [1],dtype='float32' )  # Ted doesn't like large shear. Distorts photos 
    shr = 2. * tf.random.normal( [1],dtype='float32' ) 

    h_zoom = 1.0 + tf.random.normal([1],dtype='float32') / 10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32') / 10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat( rot, shr, h_zoom, w_zoom, h_shift , w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range( DIM//2, -DIM//2, -1 ), DIM )
    y = tf.tile( tf.range( -DIM//2, DIM//2 ), [DIM] )
    z = tf.ones( [ DIM * DIM ], dtype = 'int32' )
    idx = tf.stack( [ x, y, z ] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot( m, tf.cast( idx, dtype = 'float32' ) )
    idx2 = K.cast( idx2, dtype = 'int32')
    idx2 = K.clip( idx2, -DIM//2 + XDIM + 1, DIM//2 )
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [ DIM//2 - idx2[0,], DIM//2 - 1 + idx2[1,] ] )
    d = tf.gather_nd( image, tf.transpose( idx3 ) )
        
    return tf.reshape( d, [ DIM, DIM, 3 ] ), label

In [ ]:
# https://www.kaggle.com/code/odins0n/jax-flax-tf-data-vision-transformers-tutorial
# augment by changes in Contrast, Brightness, Saturation, Rotation
def aug( data ):
    """
    Augument the image with one of four 
    different equal likely transformations.
    Transformations:
    1. Random Contrast
    2. Random Brightness
    3. Random Saturation
    4. Random Rotation
    """
    image = data["image"]
    seed = ( RANDOM_SEED, RANDOM_SEED )
    
    # two groups for transformations
    transformation_selection = tf.random.uniform( [], minval = 0, maxval = 1, dtype = tf.float32 )
    
    # probability for sub groups - each transformation has 25% chance of being applied on image
    prob_1 = tf.random.uniform( [], minval = 0, maxval = 1, dtype = tf.float32 )  
    prob_2 = tf.random.uniform( [], minval = 0, maxval = 1, dtype = tf.float32 )
    image = tf.cond( tf.greater( transformation_selection, 0.5 )
                    , lambda: tf.cond( tf.greater( prob_1, 0.5 ),
                            lambda: tf.image.stateless_random_contrast( image, 0.1, 0.5, seed = seed ), 
                            lambda: tf.image.stateless_random_brightness( image, max_delta = 0.3, seed = seed ),
                    )
                    , lambda: tf.cond( tf.greater( prob_2, 0.5 ),
                            lambda: tf.image.stateless_random_saturation( image, 0.01, 0.1, seed = seed ),
                            lambda: tfa.image.rotate( image, tf.random.uniform( (1,), minval = 0.01, maxval = 0.2 ) )                          
                    )     
    )
    data["image"] = image
    return data

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

############# ImageDataGenerator - random transformation #############

# create an ImageDataGenerator 
# update this based on image augmenation exploration results
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False
    , samplewise_center=False
    , featurewise_std_normalization=False
    , samplewise_std_normalization=False
    , zca_whitening=False
    , zca_epsilon=1e-06
    , rotation_range=36
    , width_shift_range=0.15
    , height_shift_range=0.15
    , brightness_range=None
    , shear_range = 0.0
    , zoom_range=[1.0, 1.25]
    , channel_shift_range = 0.0
    , fill_mode='constant'  # 'nearest'
    , cval = 0.0
    , horizontal_flip=True
    , vertical_flip = False
    , rescale = None
    , preprocessing_function=None
    , data_format = None
    , validation_split = 0.0
    , dtype = None
)

# define data augmentation function with random_transform method 
# for dataset.map( ... )
def img_gen_random_transform(image, label):
    # apply random_transform method to single image
    image = img_gen.random_transform(image)
    return image, label

In [ ]:
%%time
# show some 'augmentations' of a single image ( ? each epoch each training image will be augmented once )
row = 3; col = 7;
all_elements = get_train_dataset(do_aug=False).unbatch()
one_element = tf.data.Dataset.from_tensors( next(iter(all_elements)) )
augmented_element = one_element.repeat().map(transform).batch( row * col )  # ? runs out of memory

#plt.figure( figsize = ( 8 * row, 8 * col ) )
for (img,label) in augmented_element:
    plt.figure(figsize=(30,int(30*row/col)))
    for j in range(row*col):
        plt.subplot(row,col,j+1)
        plt.axis('off')
        plt.imshow(img[j,])
    plt.show()
    break
del augmented_element

In [ ]:
%%time
# get the data ( super-slow if CPU )
ds_train = get_train_dataset()

x_train = get_train_dataset_preview( ordered = True )   # x_train length is infinite
y_train = next(iter(x_train.unbatch().map(lambda image, label: label).batch(NUM_TRAIN_IMG))).numpy()
print( 'train', len(y_train), y_train )
# 12753 - array([ 93,   8,  86, ..., 102,  70,  45], dtype=int32)  # train shows up in random order
# 104 different classes, so on average ~122 of each class 

ds_valid = get_valid_dataset()  # ds_valid length is infinite
y_valid = next(iter(ds_valid.unbatch().map(lambda image, label: label).batch(NUM_VALID_IMG))).numpy()
print( 'valid', len(y_valid), y_valid )
# 3712 - array([13, 88, 53, ...,  8, 12, 67], dtype=int32) # valid is consistent order

ds_test = get_test_dataset()
#display( 'test', len( ds_test ) )
print( 'test', ds_test.cardinality().numpy() )

# can decorate something with @tf.autograph.experimental.do_not_convert to silence the warning

In [ ]:
# can we iterate over each dataset (& do something? e.g. pick colors, frequency analysis, etc.)

# EDA

In [ ]:
def batch_to_numpy( data ):
    '''Converts batch of data to numpy '''
    image , label = data
    #print( 'batch_to_numpy() label', label, label.numpy() )
    image = image.numpy()
    label = label.numpy()
    if label.dtype == object:
        label = [None for _ in enumerate(label)]
    return image , label

In [ ]:
# https://www.kaggle.com/code/lovemm/petals-to-the-metal
# what does it mean if title is blank
def title_from_label_and_target( label, correct_label ):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_batch_images( databatch, predictions = None ):
    '''Plots Some Train , Test , Validation data'''
    # load data as numpy 
    img, labels = batch_to_numpy( databatch )
    if labels is None:
        labels = [None for _ in enumerate(images)]
    
    rows = int( math.sqrt( len( img ) ) )
    cols = len(img)//rows
    # size and spacing
    FIGSIZE = 16.0
    SPACING = 0.11
    subplot = ( rows, cols, 1 )
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
        
    for i , (image, label) in enumerate(zip(img[:rows*cols],labels[:rows*cols])):
        #title = '** DEFAULT **'
        title = '?? None' if label is None else f'L:{label} - {CLASSES[label]}'
        #print( label, CLASSES[label], title )
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target( predictions[i], label )
        plt.subplot(rows,cols,i+1)
        plt.axis('off')
        plt.imshow(image)
        if title:  # was if label but label could be 0
            plt.title( title, fontsize = 9, color = 'black' if correct else 'red'  )  # CLASSES[label]
        plt.tight_layout()
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()
        

In [ ]:
import scipy.cluster
import sklearn.cluster
import numpy
from PIL import Image

def dominant_colors( image, cluster_count = 10, verbose = 0 ):  # PIL image input
    # this counts the most common colors but we want the most-important colors

    #image = image.resize((150, 150))      # optional, to reduce time
    ar = numpy.asarray(image)  # convert input to a 1D array
    shape = ar.shape
    ar = ar.reshape( numpy.product( shape[:2] ), shape[2] ).astype(float)
    print( shape, ar.shape )  # might convert from (192,192,3) to (36864,3 ) e.g. from 2D to 1D

    kmeans = sklearn.cluster.MiniBatchKMeans(
        n_clusters = cluster_count,
        init = "k-means++",
        max_iter = 20,
        random_state = 1000,
        verbose = verbose
    ).fit(ar)
    codes = kmeans.cluster_centers_
    print( 'codes', codes )
    
    myBirch = sklearn.cluster.Birch(
        n_clusters = cluster_count
    ).fit( ar )
    codes2 = myBirch.subcluster_centers_
    print( 'codes2', codes2 )

    vecs, _dist = scipy.cluster.vq.vq( ar, codes )         # assign codes
    counts, _bins = numpy.histogram( vecs, len(codes) )    # count occurrences
    print( 'counts, _bins', counts, _bins )

    colors = []
    for index in numpy.argsort( counts )[::-1]:
        #colors.append( tuple( [ int(code) for code in codes[index]]))
        colors.append( tuple( [ int( 255 * code ) for code in codes[index]]))
    return colors                    # returns colors in order of dominance

In [ ]:
# analyze a single image
# probably should convert to HSV space first & then perhaps to 1+ dimension ( H+ )
# also need to recognize that colors hues are cyclic number
image, label = batch_to_numpy( next( iter( ds_train.unbatch().batch( 1 ) ) ) )
print( label )
plt.imshow( image[0] ) # Invalid shape (1, 192, 192, 3) for image data
plt.show()
mydominant_colors = dominant_colors( image[0], 7 )
print( mydominant_colors )

palette = np.array(mydominant_colors)[np.newaxis, :, :]

plt.imshow(palette)
plt.axis('off')
plt.show()

In [ ]:
display_batch_images( next( iter( ds_train.unbatch().batch( 6 * 6 ) ) ) )
# training data is 'augmented'

In [ ]:
display_batch_images( next( iter( ds_valid.unbatch().batch( 6 * 6 ) ) ) )
# validation data is not augmented

In [ ]:
display_batch_images( next( iter( ds_test.unbatch().batch( 6 * 6 ) ) ) )

In [ ]:
import seaborn as sns
# turn this horizontonally & show both ( as percentages )

train_agg = np.asarray([[label, (y_train == index).sum()] for index, label in enumerate(CLASSES)])
valid_agg = np.asarray([[label, (y_valid == index).sum()] for index, label in enumerate(CLASSES)])
#print( train_agg )  # seems to be array of pairs, label, count ( as a string )
#print( valid_agg )
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(24, 64))

count_vals = [ eval(i) for i in train_agg[...,1] ]
label_vals = train_agg[...,0]
ax1 = sns.barplot(x=count_vals, y= label_vals , order=CLASSES, ax=ax1)
ax1.set_title('Train', fontsize=20)
ax1.tick_params( labelsize=10 )

count_vals = [ eval(i) for i in valid_agg[...,1] ]
label_vals = valid_agg[...,0]
ax2 = sns.barplot(x=count_vals, y= label_vals, order=CLASSES, ax=ax2)
ax2.set_title('Validation', fontsize=20)
ax2.tick_params( labelsize=10 )

plt.show()

# ! Data is multi-class and unbalanced ( imbalance ) ( if train_data is augmented with tf-flower-photo-tfrec, then train distribution <> valid distribution )
* Two common techniques
 * Class Weights - classifier gives greater weight to labels in minority classes
 * Oversampling ( of minority classes )

In [ ]:
%%time

# from sklearn.utils import class_weight
# my_class_weights = class_weight.compute_class_weight(
#     'balanced'
#     , np.unique(y_train)
#     , y_train
# )
# my_class_weights = dict(zip(np.unique(y_train), class_weight.compute_class_weight('balanced', np.unique(y_train), y_train))) 

weight_per_class = True

if weight_per_class:
    from collections import Counter
    import gc

    gc.enable()

    def get_training_dataset_raw():
        dataset = load_dataset(TRAINING_IMAGES, is_labeled = True, inorder = False)
        return dataset

    raw_training_dataset = get_training_dataset_raw()

    label_counter = Counter()
    for images, labels in raw_training_dataset:
        label_counter.update([labels.numpy()])

    class_count = len( label_counter )
    del raw_training_dataset
    print( 'label_counter', label_counter )  # seems to be id, count

    TARGET_NUM_PER_CLASS = 122 #??

    def get_weight_for_class(class_id):
        counting = label_counter[class_id]
        weight = TARGET_NUM_PER_CLASS / counting
        return weight

    weight_per_class = {class_id: get_weight_for_class(class_id) for class_id in range( class_count )} # dictionary of id, float
    print( weight_per_class)

In [ ]:
def create_class_weight( labels_dict, mu = 0.15 ):
    total = np.sum( list( labels_dict.values() ) )
    keys = labels_dict.keys()
    
    class_weight = dict()
    
    for key in keys:
        score = math.log( mu * total / float( labels_dict[ key ] ) )
        class_weight[key] = score if score > 1.0 else 1.0  # trims bottom ( most-common ) to 1.0
    
    return class_weight

class_count = len( label_counter )
labels_dict = {class_id: label_counter[ class_id ]  for class_id in range( class_count )}
print( labels_dict )
class_weight_dict = create_class_weight( labels_dict )
print( class_weight_dict )

In [ ]:
import seaborn as sns
from matplotlib import cm

if weight_per_class:
    df_class = pd.DataFrame.from_dict(weight_per_class, orient='index', columns=['class_weight'])
    display( df_class )
    plt.figure(figsize=(35, 6))

    #barplot color based on value
    bplot = sns.barplot(x=df_class.index, y='class_weight', data=df_class, palette= cm.Blues(df_class['class_weight']*0.15));
    for p in bplot.patches:
        bplot.annotate(format(p.get_height(), '.1f'), 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', va = 'center', 
                       xytext = (0, 9), 
                       textcoords = 'offset points')
    plt.xlabel("Class", size=14)
    plt.ylabel("Class weight (inverse of %)", size=14)

In [ ]:
if weight_per_class:
    df_class = pd.DataFrame.from_dict(class_weight_dict, orient='index', columns=['class_weight'])
    #display( df_class )
    plt.figure(figsize=(35, 6))

    #barplot color based on value
    bplot = sns.barplot(x=df_class.index, y='class_weight', data=df_class, palette= cm.Blues(df_class['class_weight']*0.15));
    for p in bplot.patches:
        bplot.annotate(format(p.get_height(), '.1f'), 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', va = 'center', 
                       xytext = (0, 9), 
                       textcoords = 'offset points')
    plt.xlabel("Class", size=14)
    plt.ylabel("Class weight (inverse of %)", size=14)

# Modeling & tuning 

# Generate Embeddings ( from a pre-trained model )
 * Notebook: Imagenet embeddings+RAPIDS SVR+Finetuned models
   * https://www.kaggle.com/code/titericz/imagenet-embeddings-rapids-svr-finetuned-models

In [ ]:
DO_EMBEDDINGS1 = False # this is a language-model embeddings pattern
if ( DO_EMBEDDINGS1 ) :
    from transformers import AutoModel, AutoTokenizer
    import torch
    import torch.nn.functional as F

In [ ]:
if ( DO_EMBEDDINGS1 ) :
    def mean_pooling( model_output, attention_mask ):
        # last_hidden_state (torch.FloatTensor of shape (batch_size, sequence_length, hidden_size)) — Sequence of hidden-states at the output of the last layer of the model.
        token_embeddings = model_output.last_hidden_state.detach().cpu()

        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand( token_embeddings.size() ).float()
        )

        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp( input_mask_expanded.sum(1), min=1e-9 )

    class EmbedDataset(torch.utils.data.Dataset):
        def __init__(self,df):
            self.df = df.reset_index(drop=True)

        def __len__(self):
            return len(self.df)

        def __getitem__(self,idx):
            text = self.df.loc[idx,"full_text"]
            tokens = tokenizer(
                    text,
                    None,
                    add_special_tokens=True,
                    padding='max_length',
                    truncation = True,
                    max_length = MAX_LEN
                , return_tensors = "pt")
            tokens = {k:v.squeeze(0) for k,v in tokens.items()}
            return tokens

    BATCH_SIZE = 4  # why this size?  This is the number of samples processed before the model is updated
# ds_tr = EmbedDataset(dftr)
# embed_dataloader_tr = torch.utils.data.DataLoader(ds_tr,\
#                         batch_size=BATCH_SIZE,\
#                         shuffle=False)
# ds_te = EmbedDataset(dfte)
# embed_dataloader_te = torch.utils.data.DataLoader(ds_te,\
#                         batch_size=BATCH_SIZE,\
#                         shuffle=False)

In [ ]:
if ( DO_EMBEDDINGS1 ) :
    tokenizer = None
    MAX_LEN = 640

    # we might instead want to get not the output of last layer but some other
    # https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/
    def get_embeddings( MODEL_NM = '', MAX = 640, BATCH_SIZE = 4, verbose = True ):
        global tokenizer, MAX_LEN
        DEVICE = "cuda"
        model = AutoModel.from_pretrained( MODEL_NM ) # output_hidden_states = True
        #model = AutoModel.from_pretrained( MODEL_NM, output_hidden_states = True ) 
        tokenizer = AutoTokenizer.from_pretrained( MODEL_NM )
        MAX_LEN = MAX

        model = model.to( DEVICE )
        model.eval() # put model in 'evaluation mode' ( feed-forward operation ) (? as compared with training/tuning mode )

        all_train_text_feats = []
        for batch in tqdm( embed_dataloader_tr, total = len( embed_dataloader_tr ) ):
            # convert to tensors
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            with torch.no_grad():
                model_output = model( input_ids = input_ids, attention_mask = attention_mask )
                #model_outputs = model( input_ids = input_ids, attention_mask = attention_mask )
                # hidden_states = model_outputs[2]
                # token_embeddings = torch.stack( hidden_states, dim = 0 ) # Concatenate the tensors for all layers. Use 'stack' to add new dimension

            sentence_embeddings = mean_pooling( model_output, attention_mask.detach().cpu() )
            # Normalize the embeddings (? why)
            sentence_embeddings = F.normalize( sentence_embeddings, p=2.0, dim=1 ) # perform Lp normalization of input over specified dimension
            sentence_embeddings = sentence_embeddings.squeeze(0).detach().cpu().numpy()
            all_train_text_feats.extend( sentence_embeddings )
        all_train_text_feats = np.array( all_train_text_feats )
        if verbose:
            print('Train embeddings shape',all_train_text_feats.shape)

        te_text_feats = []
        for batch in tqdm( embed_dataloader_te, total = len( embed_dataloader_te ) ):
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            with torch.no_grad():
                model_output = model( input_ids = input_ids, attention_mask = attention_mask )
            sentence_embeddings = mean_pooling( model_output, attention_mask.detach().cpu())
            # Normalize the embeddings
            sentence_embeddings = F.normalize( sentence_embeddings, p=2.0, dim=1 )
            sentence_embeddings = sentence_embeddings.squeeze(0).detach().cpu().numpy()
            te_text_feats.extend( sentence_embeddings )
        te_text_feats = np.array( te_text_feats )
        if verbose:
            print('Test embeddings shape',te_text_feats.shape)

        return all_train_text_feats, te_text_feats

In [ ]:
USE_timm = False
if ( USE_timm ) :
    !pip install timm  # https://timm.fast.ai/

# Use Multi-Model via timm

In [ ]:
if ( USE_timm ) :
    import timm
    avail_pretrained_models = timm.list_models( pretrained = True )
# print( len(avail_pretrained_models), avail_pretrained_models )  # 770 models

In [ ]:
if ( USE_timm ) :
    # select some subset of pretrained models
    names = [
        'deit_base_distilled_patch16_384',
        #'fbnetc_100',
        #'ig_resnext101_32x8d',
        'ig_resnext101_32x48d',
        'repvgg_b0',
        'resnetv2_152x4_bitm',
        #'rexnet_200',
        #'resnest269e',
        'swsl_resnext101_32x8d',
        #'tf_efficientnet_b6_ns',
        #'tf_efficientnet_b7_ns',
        #'tf_efficientnet_b8_ap',
        'tf_efficientnet_l2_ns_475',
        'vit_base_patch16_384',
        #'vit_large_patch16_384',
        'vit_large_r50_s32_384',
    ]

    names_hflip_crop = [
        'tf_efficientnet_l2_ns_hflip_384',
        'deit_base_distilled_patch16_384_hflip_384',
        'ig_resnext101_32x48d_hflip_384',
        'tf_efficientnet_l2_ns_512',
    ]

    names_orig = [
        'ig_resnext101_32x48d',
        'vit_large_r50_s32_384',
        'clip_RN50x4',
        'clip_ViT-B-16',
        'clip_RN50x16',
        'clip_ViT-B-32',
    ]

In [ ]:
# from glob import glob

# modelpath = { m.split('/')[-1].split('.')[0] :m for m in glob('../input/pytorch-pretrained-0/*.pt')+glob('../input/pytorch-pretrained-1/*.pt')+glob('../input/pytorch-pretrained-2/*.pt')+glob('../input/pytorch-pretrained-3/*.pt')}
# modelpath  # empty since they are not in local 'input' drive

In [ ]:
import torch
import torch.nn.functional as F

In [ ]:
if ( USE_timm ) :
    # create an individual model
    myModel = timm.create_model( 'resnet34', pretrained = True ) # num_classes = 104 + 1
    x     = torch.randn(1, 3, 224, 224)
    myModel(x).shape  # output is routinely 1000

In [ ]:
USE_fastai = False
if ( USE_fastai ) :
    from fastai.vision.all import *  # for get_image_files()

    #path = untar_data(URLs.PETS)/'images'
    # bears = DataBlock(
    #     blocks=(ImageBlock, CategoryBlock), 
    #     get_items=get_image_files, 
    #     splitter=RandomSplitter(valid_pct=0.2, seed=42),
    #     get_y=parent_label,
    #     item_tfms=Resize(128))

    # TRAINING_IMAGES = tf.io.gfile.glob( f'{GCS_PATH}/train/*.tfrec' ) # glob() returns a list of files that match the given patterns
    # VALID_IMAGES = tf.io.gfile.glob( f'{GCS_PATH}/val/*.tfrec' )
    # TEST_IMAGES = tf.io.gfile.glob( f'{GCS_PATH}/test/*.tfrec' )

    #result = get_image_files( GCS_PATH, recurse=True, folders=None ) 
    #result = get_files( f'{GCS_PATH}/train/', extensions = '.tfrec', recurse=True, folders=None ) 
    #print( result ) # []
    result = tf.io.gfile.glob( f'{GCS_PATH}/train/*.tfrec' )
    print( result )

    dls = ImageDataLoaders.from_name_func(
        GCS_PATH
        , get_image_files( GCS_PATH )  # def get_image_files(path, recurse=True, folders=None):
        #, tf.io.gfile.glob( f'{GCS_PATH}/train/*.tfrec' )
        , valid_pct=0.2
        , label_func=lambda x: x[0].isupper()
        , item_tfms = Resize(224)
    )

    # if a string is passed into the model argument, it will now use timm (if it is installed)
    learn = vision_learner(dls, 'vit_tiny_patch16_224', metrics=error_rate)

    learn.fine_tune(1)

In [ ]:
!pip install -q efficientnet

# Define custom loss functions ( used at model-compile-time )

In [ ]:
# define a custom loss function that is similar to f1 ( but f1 is not differentiable )
# since the model-submission is being evaluated on f1, best if we use it directly ( instead of sparse_categorical_accuracy )
#https://www.kaggle.com/code/rejpalcz/best-loss-function-for-f1-score-metric/notebook
    
def f1( y_true, y_pred ):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss( y_true, y_pred ):  # this seems to be just 1 - f1 ( so it is 'loss' and should be minimized )
    
#     tp = K.sum( K.cast( y_true * y_pred, 'float'), axis=0)
#     tn = K.sum( K.cast( ( 1 - y_true ) * ( 1 - y_pred ), 'float'), axis=0)
#     fp = K.sum( K.cast( ( 1 - y_true ) * y_pred, 'float'), axis=0)
#     fn = K.sum( K.cast( y_true * ( 1 - y_pred ), 'float'), axis=0)

    y_true_float = K.cast( y_true, 'float' )
    y_pred_float = K.cast( y_pred, 'float' )
    tp = K.sum( y_true_float * y_pred_float, axis=0)
    tn = K.sum( ( 1 - y_true_float ) * ( 1 - y_pred_float ), axis=0)
    fp = K.sum( ( 1 - y_true_float ) * y_pred_float, axis=0)
    fn = K.sum( y_true_float * ( 1 - y_pred_float ), axis=0)

    p = tp / ( tp + fp + K.epsilon() )
    r = tp / ( tp + fn + K.epsilon() )

    f1 = 2 * p * r / ( p + r + K.epsilon() )
    f1 = tf.where( tf.math.is_nan( f1 ), tf.zeros_like( f1 ), f1 )
    return 1 - K.mean( f1 )

In [ ]:
### Define F1 measures: F1 = 2 * (precision * recall) / (precision + recall)
# https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
# this perhaps dosen't 'average' / normalize the f1 score

def custom_f1( y_true, y_pred ):
    def recall_m( y_true, y_pred ):
        TP = K.sum( K.round( K.clip( y_true * y_pred, 0, 1)))
        Positives = K.sum( K.round( K.clip( y_true, 0, 1))) # real positives

        recall = TP / ( Positives + K.epsilon())
        return recall


    def precision_m( y_true, y_pred ):
        TP = K.sum( K.round( K.clip( y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum( K.round( K.clip( y_pred, 0, 1)))

        precision = TP / ( Pred_Positives + K.epsilon())  # ?precision could be greater than 1
        return precision

    precision, recall = precision_m( y_true, y_pred ), recall_m( y_true, y_pred )

    return 2 * ( ( precision * recall ) / ( precision + recall + K.epsilon() ) )

# can we test custom_f1() here?
# print( custom_f1( [ 0 ], [ 0 ] ) )
# print( custom_f1( [ 1 ], [ 1 ] ) )
# print( custom_f1( [ 0 ], [ 1 ] ) )
# print( custom_f1( [ 1 ], [ 0 ] ) )
# print( custom_f1( [ 1, 0 ], [ 1, 1 ] ) )

In [ ]:
a = 1
b = 0
# precision = TP / PP
# recall = TP / all_positives
# f1 = 2 * P * R / ( P + R )
print( f1_loss( torch.FloatTensor( [ a, a ] ), torch.FloatTensor( [ b, b ] ) ) ) # unsupported operand type(s) for -: 'int' and 'list'
print( f1( torch.FloatTensor( [ a, a ] ), torch.FloatTensor( [ b, b ] ) ) ) # unsupported operand type(s) for -: 'int' and 'list'
print( custom_f1( torch.FloatTensor( [ a, a ] ), torch.FloatTensor( [ b, b ] ) ) ) # unsupported operand type(s) for -: 'int' and 'list'


In [ ]:
# could define custom Callback here, then add it to callbacks in .fit()
# https://www.tensorflow.org/guide/keras/custom_callback
# would like to: perhaps stop if val_loss too bouncy
#  also predict (using curve-fit) threshold asymptope 
# perhaps predict_proba of unknown-test (after some level) & then use some trend or median
#https://neptune.ai/blog/implementing-the-macro-f1-score-in-keras
    
class myCallbackClass( tf.keras.callbacks.Callback ):
    
    def __init__(self, patience=0):
        #super(EarlyStoppingAtMinLoss, self).__init__()  # call some parent class
        self.patience = patience  # store the initialization value
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None  # initialize 
        
    #def on_train_begin(self, logs=None):
    def on_train_begin(self, logs={}):
        self._data = []
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    #def on_epoch_end(self, epoch, logs=None):
    def on_epoch_end(self, batch, logs={}):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray( self.model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        print(f' — val_f1: {val_f1} — val_precision: {val_precision}, — val_recall: {val_recall}')

        self._data.append({
            'val_rocauc': roc_auc_score(y_val, y_predict),
        })
        return
    
    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))
        print(
            "Up to batch {}, the average loss is {:7.2f}.".format(batch, logs["loss"])
        )

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))    

    def get_data(self):
        return self._data
    
myCallback = myCallbackClass()

In [ ]:
### Defining the Callback Metrics Object to track in Neptune
class myCallbackClass2( tf.keras.callbacks.Callback ):
    def __init__(self, validation, current_fold):
        #super(NeptuneMetrics, self).__init__()
        #self.exp = neptune_experiment
        self.validation = validation  # could be dataset not X_train, y_train
        self.curFold = current_fold

    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
        self.val_accuracy = []

    def on_epoch_end(self, epoch, logs={}):
        
        # something like:
        y_valid = next(iter(self.validation.unbatch().map(lambda image, label: label).batch(NUM_VALID_IMG))).numpy()
        x_valid = self.validation.map( lambda image, label: image )
        #display( 'on_epoch_end() valid: ', len( y_valid ), len( x_valid ) )
        #print( 'on_epoch_end() valid: ', len( y_valid ) )
        valid_predict = model.predict( x_valid )
        valid_preds = np.argmax( valid_predict, axis = -1 )

        decimal_places = 4
        average_method = 'macro'
        val_f1 = round( sklearn.metrics.f1_score( y_true = y_valid, y_pred = valid_preds, average = average_method ), decimal_places)
        val_recall = round( sklearn.metrics.recall_score( y_true = y_valid, y_pred = valid_preds, average = average_method ), decimal_places)
        val_precision = round( sklearn.metrics.precision_score( y_true = y_valid, y_pred = valid_preds, average = average_method ), decimal_places)
        val_accuracy = round( sklearn.metrics.accuracy_score( y_true = y_valid, y_pred = valid_preds ), decimal_places)
        # val_SparseCategoricalCrossentropy = This is a keras .fit metric, not a function to calculate

        self.val_f1s.append(val_f1)
        self.val_recalls.append(val_recall)
        self.val_precisions.append(val_precision)
        self.val_accuracy.append( val_accuracy )

        print(f' validation: count: {len(y_valid)} — val_f1: {val_f1} — val_precision: {val_precision}, — val_recall: {val_recall}')

        ### Send the performance metrics to Neptune for tracking (new version) ###
        #self.exp['Epoch End Loss'].log(logs['loss'])
        #self.exp['Epoch End F1-score'].log(val_f1)
        #self.exp['Epoch End Precision'].log(val_precision)
        #self.exp['Epoch End Recall'].log(val_recall)

#         if self.curFold == 4:
#             ### Log Epoch End metrics values for each step in the last CV fold ###
#             msg = f' End of epoch {epoch} val_f1: {val_f1} — val_precision: {val_precision}, — val_recall: {val_recall}'
#             ### Neptune new version
#             self.exp[f'Epoch End Metrics (each step) for fold {self.curFold}'] = msg

# Define Custom Learning Rate Schedulers ( used at .fit() run-time )

In [ ]:
#import kernel_tensorflow_utils as ktu
# from https://www.kaggle.com/code/chankhavu/kernel-tensorflow-utils
from abc import ABC, abstractmethod
from tensorflow.keras.callbacks import LearningRateScheduler

class LRSchedulers:
    """
    A collection of convenient learning rate schedulers. Wrapped under this one big
    class just for convenience (since this is an utility code file and we want to import
    only one file, we will use classes as namespaces).
    
    The classes in LRSchedulers namespace SHOULD inherit from LRVisualizer and hence should
    contain a method to visualize the learning rate.
    """

    class LRVisualizer(ABC):
        """The base class for learning rate visualization"""
        
        @abstractmethod
        def _lr_by_batchnum(self, steps_per_epoch, epochs):
            pass
        
        def visualize(self, steps_per_epoch, epochs):
            learning_rates = self._lr_by_batchnum(steps_per_epoch, epochs)
            plt.plot(np.arange(steps_per_epoch * epochs), learning_rates)
            plt.xlabel('batch number')
            plt.ylabel('learning rate')
            
    class FineTuningLR(LearningRateScheduler, LRVisualizer):
        """
        Starts small (to not ruin delicate pre-trained weights), increases, then decays exponentially.

        # Arguments:
            lr_start: the initial learning rate
            lr_max: the peak learning rate
            lr_min: the lowest learning rate at the end
            lr_rampup_epochs: number of epochs before peak
            lr_sustain_epochs: number of epochs at the peak
            lr_exp_decay: exponential lr decay parameter
            verbose: int. 0: quiet, 1: update messages.
        """
        def __init__(self,
                     lr_start=0.00001,
                     lr_max=0.00005,
                     lr_min=0.00001,
                     lr_rampup_epochs=5,
                     lr_sustain_epochs=0,
                     lr_exp_decay=.8,
                     verbose=0):

            self.lr_start = lr_start
            self.lr_max = lr_max
            self.lr_min = lr_min
            self.lr_rampup_epochs = lr_rampup_epochs
            self.lr_sustain_epochs = lr_sustain_epochs
            self.lr_exp_decay = lr_exp_decay

            super().__init__(self.schedule, verbose)

        def schedule(self, epoch):
            if epoch < self.lr_rampup_epochs:
                lr = (self.lr_max - self.lr_start) / self.lr_rampup_epochs * epoch + self.lr_start
            elif epoch < self.lr_rampup_epochs + self.lr_sustain_epochs:
                lr = self.lr_max
            else:
                lr = (self.lr_max - self.lr_min) * self.lr_exp_decay ** \
                (epoch - self.lr_rampup_epochs - self.lr_sustain_epochs) + self.lr_min
            return lr

        def _lr_by_batchnum(self, steps_per_epoch, epochs):
            ret = np.zeros(shape=(steps_per_epoch * epochs))
            for epoch in range(epochs):
                ret[epoch * steps_per_epoch:(epoch + 1) * steps_per_epoch] = self.schedule(epoch)
            return ret

In [ ]:
# Adaptive LR
LR_START = 0.00005
LR_MAX = LR_START 
LR_MIN = 0.00001 
LR_RAMPUP_EPOCHS = 0 
LR_SUSTAIN_EPOCHS = 5 
LR_EXP_DECAY = 0.85

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:  
        lr = LR_START + (epoch * (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS)   
    elif epoch < (LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS):  
        lr = LR_MAX
    else:    
        lr = LR_MIN + (LR_MAX - LR_MIN) * LR_EXP_DECAY ** (epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS)

    return lr

if ( 0 == 1 ) :
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True) # starts at 5.0e-5, drops to 1.0e-5 by epoch 30
    lr = [lrfn(n) for n in range(30)]
    plt.plot(lr)
else :
    # Learning Rate Scheduler for fine-tuning jobs (first increase lr, then decrease)
    print( 'using: LRSchedulers.FineTuningLR ')
    lr_callback = LRSchedulers.FineTuningLR(
        lr_start=1e-5
        , lr_max=5e-5 * strategy.num_replicas_in_sync
        , lr_min=1e-5
        , lr_rampup_epochs=5
        , lr_sustain_epochs=0
        , lr_exp_decay=0.8
        , verbose=1
)

# Define the Model

In [ ]:
from tensorflow.keras.applications import DenseNet201
import efficientnet.tfkeras as efficientnet

#define model
def create_model( input_shape, N_CLASSES, pretrained_model_trainable = False ):
    if ( 0 == 1 ) :
        #pretrained_model = tf.keras.applications.VGG16
        #pretrained_model = tf.keras.applications.DenseNet201
        #pretrained_model = tf.keras.applications.InceptionResNetV2
        #pretrained_model = tf.keras.applications.InceptionV3
        #pretrained_model = tf.keras.applications.MobileNet
        #pretrained_model = tf.keras.applications.MobileNetV2
        #pretrained_model = tf.keras.applications.NASNetMobile
        #pretrained_model = tf.keras.applications.ResNet50
        #pretrained_model = tf.keras.applications.ResNet101V2
        #pretrained_model = tf.keras.applications.VGG19
        #pretrained_model = tf.keras.applications.Xception
        #pretrained_model = tf.keras.applications.DenseNet201 
        #pretrained_model = EfficientNetB7
        
        pretrained_model = tf.keras.applications.xception.Xception(
            include_top = False,
            weights = 'imagenet',
            input_shape = (*IMAGE_SHAPE,3)
        )
        pretrained_model.trainable = pretrained_model_trainable
        model = tf.keras.Sequential([
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense( 128, activation = 'relu' ),
            tf.keras.layers.Dense( len( CLASSES ), activation = 'softmax' )
        ])
    elif ( 1 == 0 ) :
        base_model = efn.EfficientNetB6(
            weights='/kaggle/input/efficientnet/efficientnet-b6_noisy-student_notop.h5'
            , include_top=False
            , input_shape=input_shape
        )

        base_model.trainable = pretrained_model_trainable # Freeze layers
        model = tf.keras.Sequential([
            base_model,
            L.GlobalAveragePooling2D(),
            L.Dense( len( CLASSES ), activation='softmax')
        ])
    elif ( 0 == 1 ) :  # this model is slower ( but possibly more accurate )
        # https://www.kaggle.com/code/chankhavu/a-beginner-s-tpu-kernel-single-model-0-97
        feature_extractor= efficientnet.EfficientNetB7(
            weights='noisy-student'
            , include_top=False
            , input_shape=[*IMAGE_SHAPE, 3])
        feature_extractor.trainable = pretrained_model_trainable
        model = tf.keras.Sequential([
            feature_extractor,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense( len( CLASSES ), activation = 'softmax', dtype = 'float32' )
        ], name = 'custom_EfficientNetB7_model')
    else :
        # DenseNet-201 is a convolutional neural network that is 201 layers deep.
        #  You can load a pretrained version of the network trained on more than a million images from the ImageNet database.
        #  The pretrained network can classify images into 1000 object categories, such as keyboard, mouse, pencil, and many animals. 
        base_model = DenseNet201(
            include_top = False,   # whether to include the fully-connected layer at the top of the network.
            weights = 'imagenet',
            input_shape = ( IMAGE_SHAPE[0], IMAGE_SHAPE[1], 3),  # ? causes input images to be resized for DenseNet
            #pooling = None  ? 'avg'
            #classes = 1000
        )
        #print( base_model.summary() )
        # trainable rnet
        base_model.trainable = pretrained_model_trainable # True
        dropout_rate = 0.2
        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout( rate = dropout_rate ),
            tf.keras.layers.Dense( units = len(CLASSES), activation='softmax', dtype='float32', kernel_regularizer = tf.keras.regularizers.l2() )
        ], name = 'custom_DenseNet201_model' )
    
    return model

# Model Warm-up - Compile-Time set optimizer, loss and metrics

In [ ]:
%%time
#from sklearn.metrics import f1_score, precision_score, recall_score

# ?? warmup top layers first before sub-model?
WARMUP_LEARNING_RATE = 1e-4 * strategy.num_replicas_in_sync  #REPLICAS

with strategy.scope():  # mirrored_strategy.scope():
    model = create_model( (None, None, 3), len(CLASSES) )
    myMetric1 = F1Score( num_classes = len(CLASSES), average = 'macro', name = 'F1S_macro' ) # True positivies, false positives and false negatives are computed for each class and their unweighted mean is returned.
    myMetric2 = F1Score( num_classes = len(CLASSES), average = 'micro', name = 'F1S_micro' ) # micro: True positivies, false positives and false negatives are computed globally.
    
    metric_list = [   # additional metrics beyond loss_function
        tf.keras.metrics.SparseCategoricalAccuracy( name = 'SCA' )
        #'sparse_categorical_accuracy'
        #, tf.keras.metrics.SparseCategoricalCrossentropy( name = 'SCC', from_logits = False )
        #, tf.keras.metrics.SparseCategoricalCrossentropy( name = 'SCCL', from_logits = True )
        #'sparse_categorical_accuracy'
        #, myMetric1
        #, myMetric2    
    ]

    #loss_function = tf.keras.losses.SparseCategoricalCrossentropy( name = 'SCC' )   # How often predictions match integer labels ? 1.0 is perfect
        # f1_loss

    optimizer = tf.keras.optimizers.Adam( lr = WARMUP_LEARNING_RATE )

    # ERR - ValueError: Metric (<tensorflow.python.keras.metrics.SparseCategoricalAccuracy object at 0x7fd4f78667d0>) passed to model.compile was created inside of a different distribution strategy scope than the model.
    # All metrics must be created in the same distribution strategy scope as the model (in this case <tensorflow.python.distribute.tpu_strategy.TPUStrategy object at 0x7fd5ac927b50>).
    # If you pass in a string identifier for a metric to compile the metric will automatically be created in the correct distribution strategy scope.
    model.compile(
        loss = 'sparse_categorical_crossentropy'  # loss_function
        , optimizer = optimizer
        , metrics = metric_list
    )

model.summary()  # should show model name
# custom_EfficientNetB7_model - 64.3M Total params, 266K trainable ( 2560 features post base model )

In [ ]:
# ! Data is ready

# data dump of some samples
print("Training data shapes (3):")
for ix, (image, label) in enumerate( get_train_dataset().take(3) ):
    print(ix, image[ix].numpy().shape, label[ix].numpy().shape)
print("Training data label examples:", len(label), label.numpy())

print("Validation data shapes (3):")
for image, label in get_valid_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Validation data label examples:", len(label), label.numpy())

print("Test data shapes (3):")
for image, idnum in get_test_dataset().take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", len(idnum), idnum.numpy().astype('U')) # U=unicode string

# Model Warm-up train-fit() based on learning-reate & validate ( subject to early-stopping )

In [ ]:
%%time
print(".fit() the model ( warm-up mode )")
# probably this should run ( quickly ) until it stops making significant progress
STEPS_PER_EPOCH = NUM_TRAIN_IMG // BATCH_SIZE

fit_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss'
        , mode = 'min'
        , min_delta = 0.05 # perhaps too high
        , patience = 3 # this model is not very bouncy so we don't need much patience
        , verbose = 1
        , restore_best_weights = True
    )
    , myCallbackClass2( validation = ds_valid, current_fold = 0 )
]

#distributed_ds_train = strategy.experimental_distribute_dataset( ds_train )
#distributed_ds_valid = strategy.experimental_distribute_dataset( ds_valid )

warmup_history = model.fit(
    x = ds_train  # distributed_ds_train # 
    , steps_per_epoch = STEPS_PER_EPOCH  # = ~797 = 12753 training images / BATCH_SIZE = 16 * replicas
    , validation_data = ds_valid  # distributed_ds_valid #   # ? dataset so probablly connect access [0] & [1]
    , epochs = WARMUP_EPOCHS
    , callbacks = fit_callbacks
    , verbose = 2
).history

# warm-up results : 3 Epochs in 1m18s is quite fast, extend to more Epochs while we're efficiently improving
# Epoch 1/3
# 99/99 - 55s - loss: 3.0544 - sparse_categorical_accuracy: 0.3157 - val_loss: 2.0673 - val_sparse_categorical_accuracy: 0.5533
# Epoch 2/3
# 99/99 - 11s - loss: 1.7140 - sparse_categorical_accuracy: 0.6052 - val_loss: 1.4585 - val_sparse_categorical_accuracy: 0.6724
# Epoch 3/3
# 99/99 - 10s - loss: 1.2852 - sparse_categorical_accuracy: 0.7015 - val_loss: 1.2005 - val_sparse_categorical_accuracy: 0.7276
# CPU times: user 22.5 s, sys: 1.11 s, total: 23.6 s
# Wall time: 1min 18s -> 23s / Epoch ( first Epoch takes the most time, rest take only 10s ), 10 Epochs only takes 3m so 18s/E
# with 512, takes 50s/iteration ( 5x longer )

# w/ 15/20 Epoch can get to:  loss: 0.6927 - SCA: 0.8070 - SCC: 0.6927 - SCCL: 0.6927 - val_loss: 0.7296 - val_SCA: 0.8163 - val_SCC: 0.7296 - val_SCCL: 0.7296
# Wall time: 18m 53s - 20 EPOCH - Accelerator = GPU P100 - 113s EPOCH 1, 54s for EPOCH 2+  (? 797 steps  = 12753 training images / batch_size = 16)
# 797/797 - 53s - loss: 0.8170 - SCA: 0.7914 - val_loss: 0.8172 - val_SCA: 0.8006

# Wall time: 3min 21s 99/99 - 10s - loss: 0.7145 - SCA: 0.8065 - val_loss: 0.7486 - val_SCA: 0.8144
#  validation: count: 3712 — val_f1: 0.4145 — val_precision: 0.5343, — val_recall: 0.3944

# with more training data, model doesn't do well
# Epoch 5/20 - 531/531 - 249s - loss: 2.2327 - SCA: 0.6109 - val_loss: 3.4398 - val_SCA: 0.2249
# validation: count: 3712 — val_f1: 0.1304 — val_precision: 0.3227, — val_recall: 0.1173

In [ ]:
# at this point, can we evaluate the model?
# modelLoss, modelAccuracy = model.evaluate(
#     ds_train # ds_train is infinite
# )
# print( f'Training Data - model Loss = {modelLoss}, Accuracy = {modelAccuracy}' )
modelLoss, modelAccuracy = model.evaluate(
    ds_valid
)
print( f'Validation Data - model Loss = {modelLoss}, Accuracy = {modelAccuracy}' )
# Validation Data - model Loss = 1.7844997644424438, Accuracy = 0.7211745381355286
# Validation Data - model Loss = 1.6651184558868408, Accuracy = 0.7599676847457886
# Validation Data - model Loss = 2.4005961418151855, Accuracy = 0.6427801847457886 ( seems to be like val_SCA )

# Model - Post-Warm-up - make all layers trainable

In [ ]:
with strategy.scope():  # mirrored_strategy.scope():

    for layer in model.layers:
        layer.trainable = True # Unfreeze layers

    myNadam = tf.keras.optimizers.Nadam(
        learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Nadam"
    )

    #loss_function = tf.keras.metrics.SparseCategoricalAccuracy( name='SCA', dtype=None) # 'sparse_categorical_accuracy'
    loss_function = tf.keras.losses.SparseCategoricalCrossentropy( name = 'SCA', from_logits = True, reduction = tf.keras.losses.Reduction.SUM ) # 'sparse_categorical_accuracy'

    compile_metrics = [
        tf.keras.metrics.SparseCategoricalAccuracy( name = 'SCA' )  #'sparse_categorical_accuracy' # loss_function
        , tf.keras.metrics.CategoricalAccuracy( name = 'CA' )  # 'categorical_accuracy'
        , f1 
        , custom_f1
    ]

    model.compile(
        optimizer = myNadam,   # 'adam', 'nadam'
        loss = 'sparse_categorical_crossentropy',   # f1_loss,
        metrics = compile_metrics
    )

model.summary()

In [ ]:
# base model 
# Total params: 21,137,168
# Trainable params: 21,082,640
# Non-trainable params: 54,528

# efficientnet-b7 may have 64M total parameters
# densenet201 has 18.5M Total params ( w/ no trainable becomes 199,784 trainable )

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001, # minimium amount of change to count as an improvement
    patience = 5, # how many epochs to wait before stopping
    restore_best_weights = True,
    verbose = 1
)

# in general, this model mostly gets better each iteration so this is a bit of a waste plus the model is large/heavy
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint( 'best_model.hdf5', monitor = 'sparse_categorical_accuracy', verbose = 1, save_best_only = True, mode = 'max' )

# .fit() the model

In [ ]:
%%time
# training
# add time-tracking in here
STEPS_PER_EPOCH = NUM_TRAIN_IMG // BATCH_SIZE
EPOCHS = TRAIN_EPOCHS

fit_callbacks = [ lr_callback, early_stopping, myCallbackClass2( validation = ds_valid, current_fold = 0 ) ]
if CFG.wandb :
    wandb.init( project = 'flower-classification-tpu-public', job_type = 'train', reinit = True )
    # WandbCallback will automatically log history data from any metrics collected by keras: loss and anything passed into keras_model.compile().
    #fit_callbacks = fit_callbacks + WandbCallback
        # Initialize W&B run for experiment tracking
    #wandb_run = wandb.init( entity='tnadeau', project='2022-11-Blend', job_type='train', name=f'fold_{fold}', reinit = True )
    #assert wandb.run is not None
    wandb_callback = WandbCallback() # save_model = False, log_weights = True
    fit_callbacks = [ lr_callback, early_stopping, wandb_callback ]  # save_model = True log_gradients = True # training_data argument is required for gradient logging.
    #fit_callbacks = [ early_stopping, WandbCallback( save_model = False, log_weights = True ) ]  # save_model = True log_gradients = True # training_data argument is required for gradient logging.

#fit_callbacks = [ myCallbackClass2(npt_exp, validation=(x_val, y_val), current_fold=k_fold)]

# better to move this to k-fold technique
history = model.fit(
    ds_train,
    validation_data = ds_valid,
    epochs = EPOCHS,
    steps_per_epoch = STEPS_PER_EPOCH,
    callbacks = fit_callbacks,
    class_weight = weight_per_class,  # Ted - Added
    verbose = 2
)

if CFG.wandb :
    wandb.finish()

#
# Epoch 1/15 Epoch 00001: LearningRateScheduler reducing learning rate to 5e-05.
# 99/99 [==============================] - 109s 693ms/step - loss: 4.2723 - sparse_categorical_accuracy: 0.1285 - val_loss: 2.8269 - val_sparse_categorical_accuracy: 0.3817
# Epoch 00015: LearningRateScheduler reducing learning rate to 1.9264677851328125e-05.
# 99/99 [==============================] - 48s 491ms/step - loss: 0.0549 - sparse_categorical_accuracy: 0.9953 - val_loss: 0.3277 - val_sparse_categorical_accuracy: 0.9221

# with accelerator = None ( CPU ), then 400% ( ? slow ), GPU T4 x 2 ( shows 1 GPU at 50% )
# with 512, Epoch = 50s

* Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
* 99/99 - 172s - loss: 4.6632 - sparse_categorical_accuracy: 0.0244 - categorical_accuracy: 0.0042 - f1: 0.0000e+00 - custom_f1: 78.5914 - val_loss: 4.4771 - val_sparse_categorical_accuracy: 0.0415 - val_categorical_accuracy: 0.0057 - val_f1: 0.0000e+00 - val_custom_f1: 79.2682
* 
* Epoch 00026: LearningRateScheduler reducing learning rate to 1.4496393867966709e-05.
* 99/99 - 13s - loss: 0.0116 - sparse_categorical_accuracy: 0.9953 - categorical_accuracy: 0.0214 - f1: 0.0191 - custom_f1: 1.2919 - val_loss: 0.3025 - val_sparse_categorical_accuracy: 0.9283 - val_categorical_accuracy: 0.0213 - val_f1: 0.0187 - val_custom_f1: 1.6426
* Restoring model weights from the end of the best epoch.
* Epoch 00026: early stopping

* Epoch 00022: LearningRateScheduler reducing learning rate to 2.0977524091715595e-05.
* 99/99 - 52s - loss: 0.0088 - sparse_categorical_accuracy: 0.9970 - categorical_accuracy: 0.0214 - f1: 0.0191 - custom_f1: 1.2400 - val_loss: 0.2305 - val_sparse_categorical_accuracy: 0.9531 - val_categorical_accuracy: 0.0210 - val_f1: 0.0189 - val_custom_f1: 1.3983
* Restoring model weights from the end of the best epoch.
* Epoch 00022: early stopping
* CPU times: user 4min 3s, sys: 24.4 s, total: 4min 27s
* Wall time: 24min 1s

In [ ]:
# at this point, can we evaluate the model?
# modelLoss, modelAccuracy = model.evaluate(
#     ds_train # ds_train is infinite
# )
# print( f'Training Data - model Loss = {modelLoss}, Accuracy = {modelAccuracy}' )
modelLoss, modelAccuracy, CA, f1, custom_f1 = model.evaluate(  # too many values to unpack (expected 2)
    ds_valid
)
print( f'Validation Data - model Loss = {modelLoss}, Accuracy = {modelAccuracy}' )
#Validation Data - model Loss = 0.9083672761917114, Accuracy = 0.8809267282485962
# 29/29 [==============================] - 5s 129ms/step - loss: 0.4066 - SCA: 0.9526 - CA: 0.0207 - f1: 0.0186 - custom_f1: 1.6619
# Validation Data - model Loss = 0.4065918028354645, Accuracy = 0.9525861740112305
# 29/29 [==============================] - 4s 127ms/step - loss: 0.4216 - SCA: 0.9502 - CA: 0.0210 - f1: 0.0187 - custom_f1: 1.6094
# Validation Data - model Loss = 0.4216325581073761, Accuracy = 0.9501616358757019

In [ ]:
# should be improved to be agnostic to history keys
# should show trendlines, formulas and projection ( as f( EPOCH, time ) )
def plot_hist( history, EPOCHS ):
    
    epochs_ct = len( history.history['loss'] )
    plt.subplot(2,1,1)
    loss = history.history['loss']
    vloss = history.history['val_loss']
    plt.plot(range(1,epochs_ct+1),loss,c='b',label='loss')
    plt.plot(range(1,epochs_ct+1),vloss,c='r',label='val_loss')
    plt.legend()
    
    plt.subplot(2,1,2)
    acc = history.history['SCA']
    vacc = history.history['val_SCA']
    plt.plot(range(1,epochs_ct+1),acc,c='b',label='SCA')
    plt.plot(range(1,epochs_ct+1),vacc,c='r',label='val_SCA')
    plt.legend()
    
    plt.plot()

In [ ]:
plot_hist( history, EPOCHS )
# use curve-fit to predict asymptope 

In [ ]:
# save the model
model.save('Petals_to_the_Metal_save1.h5')

# Model Evaluation - Validation on known ( train & valid )

In [ ]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score

dataset = get_train_dataset()

dataset = dataset.unbatch().batch( 9*9 )
batch = iter(dataset)

images, labels = next(batch)
probabilities = model.predict(images)
predictions = np.argmax( probabilities, axis = -1 )
print( 'predicted labels', predictions.shape, predictions )
print( 'correct labels', labels.shape, labels )

# perhaps change to 'micro'
average_method = 'micro'  # or 'macros'
score = sklearn.metrics.f1_score( labels, predictions, labels=range(len(CLASSES)), average=average_method, zero_division = 'warn' )
precision = sklearn.metrics.precision_score( labels, predictions, labels=range(len(CLASSES)), average=average_method, zero_division = 'warn')
recall = sklearn.metrics.recall_score( labels, predictions, labels=range(len(CLASSES)), average=average_method, zero_division = 'warn')

#cmat = confusion_matrix(labels, predictions, labels=range(len(CLASSES)))
#display_confusion_matrix(cmat, score, precision, recall)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall)); print()
# f1 score: 0.113, precision: 0.112, recall: 0.115
#f1 score: 0.356, precision: 0.371, recall: 0.368
#f1 score: 0.471, precision: 0.471, recall: 0.471 - tuned

display_batch_images((images, labels), predictions)
# ( shows true [, prediction])

# classification_report(s)
* Some flowers are 'most' confused
 * ? wrong prediction - to: wild geranium, common dandelion, iris, thorn apple, wild rose, 

In [ ]:
%%time
# run the model on known training set
from sklearn.metrics import classification_report

x_train = get_train_dataset_preview( ordered = True )
y_train = next(iter(x_train.unbatch().map(lambda image, label: label).batch(NUM_TRAIN_IMG))).numpy()

#train_dataset = get_training_data_preview( do_aug = False, do_repeat = False, do_shuffle = False ) 
#batch = iter(train_dataset)
#x_train, y_train = next(batch)
#x_train = train_dataset.map( lambda image, label: image )  # train is generically an 'infinite' dataset
#x_train, y_train = next( iter( train_dataset.unbatch().map(lambda image, label: label).batch(NUM_TRAIN_IMG) ) )
#x_train, y_train = next( iter( train_dataset.batch(NUM_TRAIN_IMG) ) )
#x_train = train_dataset.map( lambda image, label: image)
print( len( y_train ) ) # 128  should be 12753

train_predict = model.predict( x_train )
train_preds = np.argmax( train_predict, axis=-1)

average_method = 'macro' # macro, weighted, samples
train_precision_score, train_recall_score, train_fbeta_score, train_support_score = sklearn.metrics.precision_recall_fscore_support( y_true = y_train, y_pred = train_preds, average = average_method )
print( 'train: precision_recall_fscore_support', train_precision_score, train_recall_score, train_fbeta_score, train_support_score )

train_f1_score = sklearn.metrics.f1_score( y_train, train_preds, average = average_method )
train_precision_score = sklearn.metrics.precision_score( y_train, train_preds, average = average_method )
train_recall_score = sklearn.metrics.recall_score( y_train, train_preds, average = average_method )
train_accuracy_score = sklearn.metrics.accuracy_score( y_train, train_preds )
print( 'train: f1, precision, recall', train_f1_score, train_precision_score, train_recall_score, train_accuracy_score )

print( classification_report( y_true = y_train, y_pred = train_preds, target_names = CLASSES, digits = 3 ) )  # these should all be 1.00 ideally ( esp. on known training data )

In [ ]:
x_valid = ds_valid.map( lambda image, label: image )
valid_predict = model.predict( x_valid )
valid_preds = np.argmax( valid_predict, axis = -1 )

average_method = 'macro' # micro macro, weighted, samples
valid_f1_score = sklearn.metrics.f1_score( y_valid, valid_preds, average = average_method )
valid_precision_score = sklearn.metrics.precision_score( y_valid, valid_preds, average = average_method )
valid_recall_score = sklearn.metrics.recall_score( y_valid, valid_preds, average = average_method )
valid_accuracy_score = sklearn.metrics.accuracy_score( y_valid, valid_preds )
print( 'valid: f1, precision, recall, accuracy', valid_f1_score, valid_precision_score, valid_recall_score, valid_accuracy_score )

output_dict = classification_report( y_valid, valid_preds, target_names = CLASSES, output_dict = True )
fig, ax = plt.subplots(figsize=(6,22)) 
sns.heatmap(pd.DataFrame(output_dict).iloc[:-1, :].T, annot=True, fmt='.3f', cmap = 'viridis', ax=ax)
print( classification_report( y_valid, valid_preds, target_names = CLASSES, digits = 3 ) )

In [ ]:
%%time
dataset = get_valid_dataset()

dataset = dataset.unbatch().batch(8*8)  # this only gets 20, we want them all, without it, we only get 16
batch = iter(dataset)

images, labels = next( batch )

probabilities = model.predict( images )
predictions = np.argmax( probabilities, axis = -1 )
print( 'predicted labels', predictions.shape, predictions )
print( 'correct labels', labels.shape, labels )
display_batch_images((images, labels), predictions)


if ( 0 == 1 ) :
    dataset = dataset.unbatch().batch( NUM_VALID_IMG )  # this only gets 20, we want them all, without it, we only get 16
    # 2022-12-01 20:01:47.243723: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1642070016 exceeds 10% of free system memory.
    batch = iter(dataset)
    images, labels = next( batch )

probabilities = model.predict( images )
predictions = np.argmax( probabilities, axis = -1 )

average_method = 'micro'  # or 'macros'
score = f1_score( labels, predictions, labels=range(len(CLASSES)), average=average_method, zero_division = 'warn' )
precision = precision_score( labels, predictions, labels=range(len(CLASSES)), average=average_method, zero_division = 'warn')
recall = recall_score( labels, predictions, labels=range(len(CLASSES)), average=average_method, zero_division = 'warn')

#cmat = confusion_matrix(labels, predictions, labels=range(len(CLASSES)))
#display_confusion_matrix(cmat, score, precision, recall)
print('validation count = {:d} f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(len(labels), score, precision, recall)); print()
# ? f1 score: 0.948, precision: 0.952, recall: 0.945
# f1 score: 0.113, precision: 0.112, recall: 0.115
# 3712 - f1 score: 0.687, precision: 0.672, recall: 0.746
# validation count = 100 f1 score: 0.336, precision: 0.354, recall: 0.352
# validation count = 100 f1 score: 0.432, precision: 0.436, recall: 0.432
# validation count = 100 f1 score: 0.950, precision: 0.950, recall: 0.950

# we should show:
#  images that don't work
#  confidence levels
# 1 black-eye Susan was mis-categorized as sunflower

In [ ]:
# mis-matches on validation data
# https://www.kaggle.com/code/georgezoto/computer-vision-petals-to-the-metal#Step-3:-Loading-the-Competition-Data
mismatches = sum( predictions != labels.numpy() )  # labels is a tensor, predictions is a list
NUM_VALIDATION_IMAGES = len( predictions )
print('Number of mismatches (mis-categorization) on validation data: {} out of {} or ({:.2%})'.format(mismatches, NUM_VALIDATION_IMAGES, mismatches/NUM_VALIDATION_IMAGES))

In [ ]:
%%time
from sklearn.metrics import confusion_matrix  # (y_true, y_pred, *, labels=None, sample_weight=None, normalize=None)[source]¶
from sklearn.metrics import ConfusionMatrixDisplay # (confusion_matrix, *, display_labels=None)[source]¶

cmat = confusion_matrix(
    labels.numpy(),
    predictions,
    labels = labels,
)
disp = ConfusionMatrixDisplay( cmat )  # is large 104 x 104
fig, ax = plt.subplots(figsize=(25,25))
disp.plot( ax = ax )
plt.show()
#display_confusion_matrix(cmat, score, precision, recall)

In [ ]:
# valid_acc_jnp, valid_preds_jnp, valid_true_jnp = model.evaluate(  
#     ds_valid , 
#     #NUM_VALIDATION_IMAGES , 
#     BATCH_SIZE
# )

# val_acc = valid_acc_jnp.item()*100
# print(f"Validation Accuracy = {val_acc:.2f}%")

# Predication on unknown test, then submission

In [ ]:
#load the test dataset
test_ds = get_test_dataset(ordered=True)

In [ ]:
# seperate image and id 
test_images_ds = test_ds.map(lambda image, idnum: image)
test_id_ds = test_ds.map(lambda image, idnum: idnum)

In [ ]:
%%time
# perform prediction on the test data using trained model
pred = model.predict( test_images_ds )  # pred is (7382, 104)
print( pred.shape, pred )# an array of 7382 arrays each with 104 values which are the ? relative probabilities )

# since the model output is sparse we only need the max value index
# would also like to know confidence
label_pred = np.argmax( pred, axis = -1 )

In [ ]:
# display some predictions on unknown/test
# display some that we are quite sure of
# display some that we are unsure of
#display_batch_images((images, labels), pred)

In [ ]:
df_pred = pd.DataFrame( pred )
df_pred['max'] = df_pred.aggregate( ['max'], axis = 1)
df_pred['label'] = label_pred
display( df_pred )
display( df_pred.describe() ) # every col ( class ) has a max of near 1.0 ( which is the one that it is most likely )

In [ ]:
# some rows that we are pretty sure of
#display( df_pred.iloc[[4,1502,1989,7378,7379,7380]] )  # 46 #58 70
#display( df_pred[ df_pred['max'] > 0.999 ] ) # >0.999 3781 rows of 7382
#display( df_pred[ df_pred['max'] > 0.99 ] ) # >0.99 5318 rows of 7382
#display( df_pred[ df_pred['max'] > 0.95 ] ) # >0.95 6072 rows of 7382
#display( df_pred[ df_pred['max'] > 0.80 ] ) # >0.80 6676 rows of 7382
#display( df_pred[ df_pred['max'] > 0.70 ] ) # >0.70 6841 rows of 7382
display( df_pred[ df_pred['max'] > 0.50 ] ) # >0.70 7178 rows of 7382, 7230, 7166

In [ ]:
# some rows that are troubling ( confidence is low )
display( df_pred.iloc[[4,1502,1989,7378,7379,7380]] )  # 46 #58 70


In [ ]:
display( df_pred[ df_pred['max'] < 0.30 ] )

In [ ]:
%%time
# Id dataset -> numpy 
ids = next(iter(test_id_ds.unbatch().batch(NUM_TEST_IMG))).numpy().astype('U')

# dictionary to create dataframe from data
submission_dict = {
    'id':ids,
    'label':label_pred
}
df_submission = pd.DataFrame(submission_dict)
display( df_submission.head(10) )
display( df_submission.tail(10) )

# 	id	label
# 0	252d840db	67
# 1	1c4736dea	28 (? wrong is 4)
# 2	c37a6f3e9	83  # possibly 77 ?81, ? wrong is 103
# 3	00e4f514e	103
# 4	59d1b6146	46 #58 70 ( ? wrong is 67)
# 5	8d808a07b	53
# 6	aeb67eefb	52 ( wrong is 103 )
# 7	53cfc6586	29 48 ( ? wrong is 49 )
# 8	aaa580243	82 ( ? wrong is 67 )
# 9	d907ca7c0	13

# id	label
# 7372	298ade3a4	49
# 7373	8361401fa	45 47
# 7374	3522d5b4e	47
# 7375	f65475a24	48
# 7376	ce3c158fa	74
# 7377	c785abe6f	7
# 7378	9b9c0e574	68 81 103
# 7379	e46998f4d	53 103 48 86
# 7380	523df966b	102 49
# 7381	e86e2a592	62

In [ ]:
#train_agg = np.asarray([[label, (y_train == index).sum()] for index, label in enumerate(CLASSES)])
valid_agg = np.asarray( [[label, ( y_valid == index ).sum() ] for index, label in enumerate(CLASSES)] )
test_agg = np.asarray( [[label, ( df_submission['label'] == index).sum() ] for index, label in enumerate(CLASSES)] )
#print( test_agg )

fig, (ax1, ax2) = plt.subplots( 2, 1, figsize=(24, 64) )

# ax1 = sns.barplot(x=train_agg[...,1], y=train_agg[...,0], order=CLASSES, ax=ax1)
# ax1.set_title('Train', fontsize=30)
# ax1.tick_params(labelsize=16)

x_vals = [ eval(i) for i in valid_agg[...,1] ]
ax1 = sns.barplot( x = x_vals, y = valid_agg[...,0], order = CLASSES, ax=ax1)
ax1.set_title('Validation', fontsize=30)
ax1.tick_params( labelsize = 16)

x_vals = [ eval(i) for i in test_agg[...,1] ]
ax2 = sns.barplot( x = x_vals, y = test_agg[...,0], order = CLASSES, ax = ax2)
ax2.set_title( 'Test', fontsize = 30 )
ax2.tick_params( labelsize = 16 )

plt.show()

In [ ]:
# save as csv
df_submission.to_csv('submission.csv',index=False)

In [ ]:
print( "Local (NY) Time: ", dt.now( tz_NY ).strftime("%Y-%m-%d %H:%M:%S") )